# 220

Contains Duplicate 应该可以在看过的系列题里面排倒数。

In [ ]:
class Solution:
    def containsNearbyAlmostDuplicate(self, nums: List[int], k: int, t: int) -> bool:
        if k < 1 or t < 0:
            return False
        dic = collections.OrderedDict()
        for n in nums:
            key = n if not t else n // t
            for m in (dic.get(key - 1), dic.get(key), dic.get(key + 1)):
                if m is not None and abs(n - m) <= t:
                    return True
            if len(dic) == k:
                dic.popitem(False)
            dic[key] = n
        return False

这个 Solution 嘛…可能需要一点额外解释：

如果： | nums[i] - nums[j] | <= t 式a

等价： | nums[i] / t - nums[j] / t | <= 1 式b

推出： | floor(nums[i] / t) - floor(nums[j] / t) | <= 1 式c

等价： floor(nums[j] / t) ∈ {floor(nums[i] / t) - 1, floor(nums[i] / t), floor(nums[i] / t) + 1} 式d

其中式b是式c的充分非必要条件，因为逆否命题与原命题等价，所以：

如果： floor(nums[j] / t) ∉ {floor(nums[i] / t) - 1, floor(nums[i] / t), floor(nums[i] / t) + 1} 非d

推出： | nums[i] - nums[j] | > t 非 a

虽然上面这种方式套了两个 for 循环，但实际上里面那个 for 循环的内容是一定的（只有三项），因此可以认定这一层循环不会增加复杂度。

事实上还有一个完全等价，但是更易懂的方法。那就是使用 BucketSort（桶排序！真没想到还能碰见这个）

In [ ]:
def containsNearbyAlmostDuplicate(self, nums, k, t):
    # Bucket sort. Each bucket has size of t. For each number, the possible
    # candidate can only be in the same bucket or the two buckets besides.
    # Keep as many as k buckets to ensure that the difference is at most k.
    buckets = {}
    for i, v in enumerate(nums):
        # t == 0 is a special case where we only have to check the bucket
        # that v is in.
        bucketNum, offset = (v / t, 1) if t else (v, 0)
        for idx in xrange(bucketNum - offset, bucketNum + offset + 1):
            if idx in buckets and abs(buckets[idx] - nums[i]) <= t:
                return True

        buckets[bucketNum] = nums[i]
        if len(buckets) > k:
            # Remove the bucket which is too far away. Beware of zero t.
            del buckets[nums[i - k] / t if t else nums[i - k]]

    return False

因为你要求了差值小于 t 且距离小于 k 的才计入之中对吧，那么事情简单了，我们只需要回溯其中一部分判断是否重复就可以了。
也就是我们的前一个和后一个 bucket。